<a href="https://colab.research.google.com/github/rithikkulkarni/ALDA-Course-Project/blob/main/finbert_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial preprocessing code

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rithikkulkarni/ALDA-Course-Project/refs/heads/main/datasets/stock_tweets.csv')

In [ ]:
df.info()

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert", device="cuda")

In [ ]:
### Function for using FinBERT to label our data for supervised learning
def label_data(tweet_text):
  temp = pipe(tweet_text)
  if 'positive' in str(temp[0]):
    return 'positive'
  elif 'negative' in str(temp[0]):
    return 'negative'
  else:
    return 'neutral'

### Function for using FinBERT that should be faster because it process multiple tweets in parallel
def label_data_batch(tweet_texts, batch_size=32):
    results = pipe(tweet_texts, batch_size=batch_size)
    labels = []
    for res in results:
        if 'positive' in str(res):
            labels.append('positive')
        elif 'negative' in str(res):
            labels.append('negative')
        else:
            labels.append('neutral')
    return labels

In [ ]:
# import numpy as np

# batch_size = 32  # Adjust based on memory and speed
# tweet_batches = np.array_split(df['Tweet'].to_numpy(), len(df) // batch_size + 1)
# label_batches = [label_data_batch(batch.tolist()) for batch in tweet_batches]

# # Flatten the list of label batches
# df['label'] = np.concatenate(label_batches)

import torch
from datasets import Dataset

# Ensure GPU is enabled
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("text-classification", model="ProsusAI/finbert", device=device)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Tweet']])  # Keep only relevant column

# Define a function to process tweets
def label_data_batch(batch):
    results = pipe(batch['Tweet'], batch_size=32)  # Adjust batch size
    return {'label': [res['label'].lower() for res in results]}  # Convert to lowercase

# Apply the function to the dataset
labeled_dataset = dataset.map(label_data_batch, batched=True)

# Convert back to DataFrame
df['label'] = labeled_dataset['label']

In [ ]:
df.info()

In [ ]:
df.rename(columns={'label': 'Label'}, inplace=True)
df.info()

In [ ]:
df.Label.value_counts()

In [ ]:
### To be used for stock price

import yfinance as yf

ticker = yf.Ticker("AAPL")
hist = ticker.history(start="2024-02-05", end="2024-02-12")
print(hist)